# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [73]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
from bson.decimal128 import Decimal128

In [74]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [75]:
# confirm that our new database was created
print(mongo.list_database_names())


['admin', 'classDB', 'config', 'fruits_db', 'local', 'met', 'tavel_db', 'uk_food']


In [76]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [77]:
# review the collections in our new database
print(db.list_collection_names())


['establishments']


In [78]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [79]:
# Create a dictionary for the new restaurant data
new_restaurant= {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}


In [80]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

In [81]:
# Check that the new restaurant was inserted
# Define the query
query = {"BusinessName": "Penang Flavours"}

restaurant = establishments.find_one(query)

# If the restaurant was found, print it
if restaurant is not None:
    print("Found the restaurant:")
    pprint(restaurant)
else:
    print("Did not find the restaurant")



Found the restaurant:
{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64dac24699f57634942e8891'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [82]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
# Find the desired document with specific fields
document = establishments.find_one({"BusinessType": "Restaurant/Cafe/Canteen"},
                                  {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0})
business_type_id = document["BusinessTypeID"]

# Print the result
pprint(document)


{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [83]:
# Update the new restaurant with the correct BusinessTypeID
update_result = establishments.update_one(
    {"BusinessName": "Penang Flavours"}, 
    {"$set": {"BusinessTypeID": business_type_id}}
)

In [84]:
# Confirm that the new restaurant was updated
query = {"BusinessName": "Penang Flavours"}

restaurant = establishments.find_one(query)

# If the restaurant was found, print it
if restaurant is not None:
    print("Found the restaurant:")
    pprint(restaurant)
else:
    print("Did not find the restaurant")

Found the restaurant:
{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64dac24699f57634942e8891'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [85]:
# Find how many documents have LocalAuthorityName as "Dover"
count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"There are {count} documents with LocalAuthorityName as 'Dover'.")


There are 0 documents with LocalAuthorityName as 'Dover'.


In [86]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments.delete_many({"LocalAuthorityName": "Dover"})

In [87]:
# Check if any remaining documents include Dover
count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"There are {count} documents with LocalAuthorityName as 'Dover'.")

There are 0 documents with LocalAuthorityName as 'Dover'.


In [97]:
# Check that other documents remain with 'find_one'
document = establishments.find_one()

if document:
    print("There are still documents remaining in the collection.")
    pprint(document)  # Print the first document found as a sample.
else:
    print("The collection is empty.")


There are still documents remaining in the collection.
{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64cc3541f8e08bc6f06b0adb'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [101]:
# Change the data type from String to Decimal for longitude and latitude
# Query to find documents where latitude and longitude are of type string
query = {
    "geocode.latitude": {"$type": "string"},
    "geocode.longitude": {"$type": "string"}
}

# Fetch all documents that match the query
cursor = establishments.find(query)

for doc in cursor:
    try:
        # Convert latitude and longitude strings to Decimal128
        latitude_decimal = Decimal128(doc["geocode"]["latitude"])
        longitude_decimal = Decimal128(doc["geocode"]["longitude"])

        # Update the current document
        establishments.update_one(
            {"_id": doc["_id"]},
            {
                "$set": {
                    "geocode.latitude": latitude_decimal,
                    "geocode.longitude": longitude_decimal
                }
            }
        )
        print(f"Updated document with _id: {doc['_id']}")

    except Exception as e:
        print(f"Error updating document with _id: {doc['_id']}. Error: {e}")

print("Update process completed.")





Updated document with _id: 64cc3541f8e08bc6f06b0adb
Updated document with _id: 64cc3541f8e08bc6f06b0adc
Updated document with _id: 64cc3541f8e08bc6f06b0add
Updated document with _id: 64cc3541f8e08bc6f06b0ae0
Updated document with _id: 64cc3541f8e08bc6f06b0ae1
Updated document with _id: 64cc3541f8e08bc6f06b0ae2
Updated document with _id: 64cc3541f8e08bc6f06b0ae3
Updated document with _id: 64cc3541f8e08bc6f06b0ae4
Updated document with _id: 64cc3541f8e08bc6f06b0ae5
Updated document with _id: 64cc3541f8e08bc6f06b0ae6
Updated document with _id: 64cc3541f8e08bc6f06b0ae7
Updated document with _id: 64cc3541f8e08bc6f06b0ae8
Updated document with _id: 64cc3541f8e08bc6f06b0ae9
Updated document with _id: 64cc3541f8e08bc6f06b0aea
Updated document with _id: 64cc3541f8e08bc6f06b0aeb
Updated document with _id: 64cc3541f8e08bc6f06b0aec
Updated document with _id: 64cc3541f8e08bc6f06b0aed
Updated document with _id: 64cc3541f8e08bc6f06b0aee
Updated document with _id: 64cc3541f8e08bc6f06b0aef
Updated docu

Use `update_many` to convert `RatingValue` to integer numbers.

In [90]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [102]:
# Confirm that the new restaurant was updated
query = {"BusinessName": "Penang Flavours"}

restaurant = establishments.find_one(query)

# If the restaurant was found, print it
if restaurant is not None:
    print("Found the restaurant:")
    pprint(restaurant)
else:
    print("Did not find the restaurant")

Found the restaurant:
{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64dac24699f57634942e8891'),
 'geocode': {'latitude': Decimal128('51.49014200'),
             'longitude': Decimal128('0.08384000')},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


In [98]:
# Change the data type from String to Integer for RatingValue
cursor = establishments.find({
    "RatingValue": {"$type": "string", "$ne": None}
})

for document in cursor:
    try:
        new_rating = int(document["RatingValue"])
        
        establishments.update_one(
            {"_id": document["_id"]},
            {"$set": {"RatingValue": new_rating}}
        )
        print(f"Updated document with _id: {document['_id']}")

    except ValueError:
        # Handle case where string cannot be converted to an integer
        print(f"Failed to convert RatingValue for document with _id: {document['_id']}")

    except Exception as e:
        # Handle other potential errors
        print(f"Error updating document with _id: {document['_id']}. Error: {e}")

print("Update process completed.")


Update process completed.


In [103]:
# Check that the coordinates and rating value are now numbers
numeric_types = ["double", "int", "decimal"]
numeric_conditions = [{"$type": t} for t in numeric_types]

# Fetch samples for verification using the correct approach with $or
longitude_sample = establishments.find_one({"$or": [{"geocode.longitude": {"$type": t}} for t in numeric_types]})
latitude_sample = establishments.find_one({"$or": [{"geocode.latitude": {"$type": t}} for t in numeric_types]})
rating_sample = establishments.find_one({"$or": [{"RatingValue": {"$type": t}} for t in numeric_types]})

# Print the samples
print("Sample document with numeric longitude:")
pprint(longitude_sample)

print("\nSample document with numeric latitude:")
pprint(latitude_sample)

print("\nSample document with numeric RatingValue:")
pprint(rating_sample)





Sample document with numeric longitude:
{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64cc3541f8e08bc6f06b0adb'),
 'geocode': {'latitude': Decimal128('51.083812'),
             'longitude': Decimal128('1.195625')},
 'links': [{'href': 'https://api.ratings.food.gov.uk/est